In [1]:
import numpy as np
import pandas as pd

from src.preprocess.text import SentenceGetter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

from tqdm.notebook import tqdm

from itertools import chain

from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

from src.preprocess.text import sent2features, sent2labels

from sklearn_crfsuite import CRF

In [2]:
ner_dataset = pd.read_csv("/Users/Mikhail_Bulgakov/GitRepo/pos_ner_task/data/ner_dataset.csv", delimiter=',', encoding='unicode_escape')
ner_dataset = ner_dataset.fillna(method="ffill")

In [3]:
sg = SentenceGetter(ner_dataset)

In [4]:
train_data, test_data = train_test_split(sg.get_full_data(), test_size=0.2, random_state=100)

In [5]:
X_train = [sent2features(s) for s in train_data]
y_train = [sent2labels(s) for s in train_data]

X_test = [sent2features(s) for s in test_data]
y_test = [sent2labels(s) for s in test_data]

In [6]:
crf = CRF(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [7]:
crf.fit(X_train,y_train)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=10, c2=0.1,
    max_iterations=100)

In [8]:
y_test_pred = list(chain.from_iterable(crf.predict(X_test)))
y_test = list(chain.from_iterable(y_test))

In [9]:
states = list(set([i[2] for i in chain.from_iterable(train_data)]))

In [18]:
df = pd.DataFrame(confusion_matrix(y_test, y_test_pred, labels=states), index=states, columns=states)
df = df.reindex(sorted(df.columns), axis=1).sort_index()
df

,B-art,B-eve,B-geo,B-gpe,B-nat,B-org,B-per,B-tim,I-art,I-eve,I-geo,I-gpe,I-nat,I-org,I-per,I-tim,O
B-art,0,0,21,3,0,24,9,0,0,0,0,0,0,5,0,0,17
B-eve,0,17,4,4,0,11,4,4,0,0,0,0,0,2,0,2,14
B-geo,0,0,6904,26,1,290,122,6,0,0,29,0,0,43,62,9,193
B-gpe,0,0,140,2864,0,23,6,0,0,0,3,0,0,7,5,0,60
B-nat,0,0,6,0,5,7,3,0,0,0,0,0,0,0,0,1,18
B-org,0,0,604,19,0,2681,216,7,0,0,3,1,0,54,70,5,260
B-per,0,0,263,2,0,161,2552,0,0,0,6,0,0,104,141,4,138
B-tim,0,0,63,2,0,11,7,3399,0,0,2,0,0,5,5,59,578
I-art,0,0,0,0,0,3,1,1,0,0,5,0,0,33,8,0,5
I-eve,0,4,1,0,0,1,1,1,0,8,5,0,0,15,5,3,14


In [19]:
df = pd.DataFrame(precision_recall_fscore_support(y_test, y_test_pred, labels=states), index=["precision", "recall", "f1_score", "support"], columns=states).round(2)
df = df.reindex(sorted(df.columns), axis=1).sort_index()
df

/Users/Mikhail_Bulgakov/opt/anaconda3/envs/nlp-course/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,B-art,B-eve,B-geo,B-gpe,B-nat,B-org,B-per,B-tim,I-art,I-eve,I-geo,I-gpe,I-nat,I-org,I-per,I-tim,O
f1_score,0.0,0.41,0.86,0.94,0.22,0.72,0.79,0.87,0.0,0.24,0.77,0.42,0.0,0.75,0.85,0.73,0.99
precision,0.0,0.81,0.83,0.95,0.83,0.77,0.82,0.92,0.0,1.00,0.80,0.92,0.0,0.74,0.80,0.82,0.99
recall,0.0,0.27,0.90,0.92,0.12,0.68,0.76,0.82,0.0,0.14,0.73,0.28,0.0,0.76,0.90,0.66,0.99
support,79.0,62.00,7685.00,3108.00,40.00,3920.00,3371.00,4131.00,56.0,58.00,1500.00,40.00,12.0,3370.00,3367.00,1329.00,178824.00


In [14]:
import eli5

In [15]:
eli5.show_weights(crf, top=30)

From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,3.909,2.15,0.0,1.694,0.0,2.1,0.0,1.681,0.0,1.703,0.0,2.919,0.0,3.881,0.0,2.546,0.0
B-art,0.0,0.0,8.412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-art,-0.383,0.0,8.051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-eve,0.0,0.0,0.0,0.0,8.234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-eve,-0.0,0.0,0.0,0.0,7.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-geo,0.93,0.0,0.0,0.0,0.0,0.0,9.26,0.676,0.0,0.0,0.0,0.428,0.0,0.0,0.0,1.877,0.0
I-geo,-0.046,0.0,0.0,0.0,0.0,0.0,7.615,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.137,0.0
B-gpe,0.86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.864,0.0,0.0,1.547,0.0,0.923,0.0,0.0,0.0
I-gpe,-0.067,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-nat,-0.255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.292,0.0,0.0,0.0,0.0,0.0,0.0
